# Import library

In [1]:
import sys
sys.path.append('src')
import en_system_ex_analysis as enex
from SALib.sample import saltelli
from SALib.analyze import sobol
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import seaborn as sns
from scipy.stats import norm, uniform, triang, gamma
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")

Load colors...
Load colormaps...


# Primary energy use, CO2 emission, Exergy efficiency

## system setting

In [ ]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

EB       = enex.ElectricBoiler()
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_sup_tap = MWF
EB.system_update()

GB = enex.GasBoiler()
GB.r0    = 0.2
GB.H     = 0.8
GB.x_ins = 0.05
GB.dV_w_sup_tap = MWF
GB.system_update()

HPB = enex.HeatPumpBoiler()
HPB.r0    = 0.2
HPB.H     = 0.8
HPB.x_ins = 0.05
HPB.dV_w_sup_tap = MWF
HPB.system_update()

Mean water flow rate: 1.11 L/min


AttributeError: 'GasBoiler' object has no attribute 'dV_w_sup_comb'

In [ ]:

# Primary energy factor
PEF_elec = 2.75 
PEF_NG = 1.1

# CO2 emission factor
Wh2MJ = 10**6/(1*enex.h2s) # enex.h2s = 3600 s/h
C_to_CO2 = 44/12 # 탄소 배출량당 이산화탄소 배출량 분자량에 의한 환산계수 
CF_NG_ori = 15.281 # tC/TJ = kgC/GJ = gC/MJ

CF_elec = 0.4747 # tCO2/MWh = gCO2/Wh
CF_NG = (CF_NG_ori*C_to_CO2)/Wh2MJ # gCO2/Wh

# Energy use
energy_use = [
        EB.energy_balance['hot water tank']['in']['E_heater'],
        GB.energy_balance['combustion chamber']['in']['E_NG'],
        HPB.energy_balance['refrigerant loop']['in']['E_cmp'] + HPB.energy_balance['external unit']['in']['E_fan']
]

# Exergy use
exergy_use = [
        EB.exergy_balance['hot water tank']['in']['E_heater'],
        GB.exergy_balance['combustion chamber']['in']['X_NG'],
        HPB.exergy_balance['refrigerant loop']['in']['E_cmp'] + HPB.exergy_balance['external unit']['in']['E_fan']
]

# Primary energy use
primary_energy_use = [
        energy_use[0] * PEF_elec,
        energy_use[1] * PEF_NG,
        energy_use[2] * PEF_elec,
]

# CO2 emission
CO2_emission = [
        energy_use[0] * CF_elec, 
        energy_use[1] * CF_NG,
        energy_use[2] * CF_elec
]

eX_efficiency = np.array([
        EB.X_eff, GB.X_eff, HPB.X_eff
]) * 100

# Create figure and axis - adjust figure size for vertical layout
nrows = 1
ncols = 4
fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(16), dm.cm2in(3.5)), dpi=200, facecolor=None, edgecolor='k')

for ridx in range(nrows):
    for cidx in range(ncols):
        idx    = ridx * ncols + cidx
        data   = [energy_use, primary_energy_use, CO2_emission, eX_efficiency][idx]
        heading   = ['Energy use [W]', 'Primary energy use [W]', 'CO_2 emission [gCO$_2$/h]', 'Exergy efficiency [%]'][idx]
        labels = [['EB', 'GB', 'HPB'], # ['EB\n(E_heater)', 'GB\n(E_NG})', 'HPB\n(E_fan} + E_cmp})']
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB']][idx]

        group_margin = 0.15  # margin between groups
        bar_margin   = 0.1  # margin between bars
        bar_width    = 0.25

        dN     = len(data)  # number of data
        x_pos0 = np.array([i * (bar_margin + bar_width) for i in range(dN)]) # 바 margin과 bar width를 곱해서 그룹 마진이 없는 x_pos0에 저장
        gl1    = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]][idx]  # group
        gN     = len(gl1)  # number of groups
        gl2    = np.cumsum(gl1)

        x_pos = x_pos0[:gl2[0]].tolist()
        for i in range(gN - 1):  # Add group_margin between arrays
            x_pos.extend((x_pos0[gl2[i]:gl2[i+1]] + group_margin * (i + 1)).tolist()) # 그룹 마진을 추가

        # Plot parameters
        sp_ratio = 0.5
        xmar = bar_width
        xmax = max(x_pos)
        xmin = min(x_pos)
        ymar = 0
        ymax = [9000, 9000, 1500, 30][idx]
        ymin = [0, 0, 0, 0][idx]
        yint = [3000, 3000, 500, 10][idx]

        # Create vertical bars
        bars = ax[idx].bar(x_pos, data, bar_width, color=[['dm.red7', 'dm.red5','dm.red3'],
                                                          ['dm.orange7', 'dm.orange5','dm.orange3'],
                                                          ['dm.green7', 'dm.green5','dm.green3'],
                                                          ['dm.grape7', 'dm.grape5','dm.grape3']][idx])

        # Annotation
        ax[idx].annotate(f'{heading}', xy=(0.5, 1.1), xycoords='axes fraction', color='dm.gray9',
                        horizontalalignment='center', verticalalignment='bottom', fontsize=dm.fs(0), fontweight=400)

        # Add value labels on top of each bar
        for bidx, bar in enumerate(bars):
            height = bar.get_height()
            if cidx < 3:
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                        f'{height:.0f}', ha='center', va='bottom', fontsize=dm.fs(-2))
            elif cidx == 3:
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                        f'{height:.1f}', ha='center', va='bottom', fontsize=dm.fs(-2))
            
        # Axes limits
        ax[idx].set_xlim(xmin - xmar, xmax + xmar)
        ax[idx].set_ylim(ymin - ymar, ymax + ymar)


        # Tick parameters
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='major', length=0, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='minor', length=0, width=0.25 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='in', labelsize=dm.fs(-1.5),
                            which='major', length=2, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='in', labelsize=dm.fs(-1.5),
                            which='minor', length=1, width=0.25 * sp_ratio, pad=3)

        # Labels and ticks
        ax[idx].set_xticks(x_pos)
        ax[idx].set_yticks(np.arange(ymin,ymax+yint,yint))  # No y-ticks
        ax[idx].set_xticklabels(labels, ha='center', fontsize=dm.fs(-1.5), fontweight='regular')
        ax[idx].set_yticklabels(np.arange(ymin,ymax+yint,yint), fontsize=dm.fs(-1.5), fontweight='regular')
        
        # Minor ticks
        # ax[idx].xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        # ax[idx].yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

        # Spines
        for k in ['top','right']:  # Only keep bottom spine
            ax[idx].spines[k].set_visible(False)
        ax[0].spines['bottom'].set_linewidth(0.5 * sp_ratio)
        ax[0].spines['bottom'].set_color('k')

plt.subplots_adjust(wspace=0.3)
dm.simple_layout(fig, bbox=(0.01,1,0.0,.945), verbose=False)
# img_stem = f'../figure/heat_pump_boiler_exergy_results'
# plt.savefig(f'{img_stem}.png', dpi=600)
dm.util.save_and_show(fig)


## GB, HPB

In [13]:
EB  = enex.ElectricBoiler()
GB  = enex.GasBoiler()
HPB = enex.HeatPumpBoiler()
EB.system_update()
GB.system_update()
HPB.system_update()

# Primary energy factor
PEF_elec = 2.75 
PEF_NG = 1.1

# CO2 emission factor
Wh2MJ = 10**6/(1*enex.h2s) # enex.h2s = 3600 s/h
C_to_CO2 = 44/12 # 탄소 배출량당 이산화탄소 배출량 분자량에 의한 환산계수 
CF_NG_ori = 15.281 # tC/TJ = kgC/GJ = gC/MJ

CF_elec = 0.4747 # tCO2/MWh = gCO2/Wh
CF_NG = (CF_NG_ori*C_to_CO2)/Wh2MJ # gCO2/Wh

# Energy use
energy_use = [
    GB.energy_balance['combustion chamber']['in']['$E_NG'],
    HPB.energy_balance['refrigerant loop']['in']['$E_cmp'] + HPB.energy_balance['external unit']['in']['$E_fan']
]

# Exergy use
exergy_use = [
    GB.exergy_balance['combustion chamber']['in']['$X_NG'],
    HPB.exergy_balance['refrigerant loop']['in']['$E_cmp'] + HPB.exergy_balance['external unit']['in']['$E_fan']
]

# Primary energy use
primary_energy_use = [
    energy_use[0] * PEF_NG,
    energy_use[1] * PEF_elec,
]

# CO2 emission
CO2_emission = [
    energy_use[0] * CF_NG,
    energy_use[1] * CF_elec
]

eX_consum = [
    calculate_total_exergy_consumption(GB.exergy_balance),
    calculate_total_exergy_consumption(HPB.exergy_balance),
]

eX_efficiency = [
    GB.exergy_balance['mixing valve']['out']['$X_w,serv']/exergy_use[0]*100,
    HPB.exergy_balance['mixing valve']['out']['$X_w,serv']/exergy_use[1]*100
]

# Create figure and axis - adjust figure size for vertical layout
nrows = 1
ncols = 4
fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(12), dm.cm2in(3.5)), dpi=200, facecolor=None, edgecolor='k')

for ridx in range(nrows):
    for cidx in range(ncols):
        idx    = ridx * ncols + cidx
        data   = [energy_use, primary_energy_use, CO2_emission, eX_efficiency][idx]
        heading   = ['Energy use [W]', 'Primary energy use [W]', 'CO$_2$ emission [gCO$_2$/h]', 'Exergy efficiency [%]'][idx]
        labels = [['GB', 'HPB'], 
            ['GB', 'HPB'],
            ['GB', 'HPB'],
            ['GB', 'HPB']][idx]

        group_margin = 0  # margin between groups
        bar_margin   = 0.05  # margin between bars
        bar_width    = 0.25

        dN     = len(data)  # number of data
        x_pos0 = np.array([i * (bar_margin + bar_width) for i in range(dN)]) # 바 margin과 bar width를 곱해서 그룹 마진이 없는 x_pos0에 저장
        gl1    = [[1,1],[1,1],[1,1],[1,1]][idx]  # group
        gN     = len(gl1)  # number of groups
        gl2    = np.cumsum(gl1)

        x_pos = x_pos0[:gl2[0]].tolist()
        for i in range(gN - 1):  # Add group_margin between arrays
            x_pos.extend((x_pos0[gl2[i]:gl2[i+1]] + group_margin * (i + 1)).tolist()) # 그룹 마진을 추가

        # Plot parameters
        sp_ratio = 0.5
        xmar = bar_width
        xmax = max(x_pos)
        xmin = min(x_pos)
        ymar = 0
        ymax = [4000, 4000, 800, 30][idx]
        ymin = [0, 0, 0, 0][idx]
        yint = [1000, 1000, 200, 10][idx]

        # Create vertical bars
        bars = ax[idx].bar(x_pos, data, bar_width, color=[['dm.red7', 'dm.red5'],
                                ['dm.orange7', 'dm.orange5'],
                                ['dm.green7', 'dm.green5'],
                                ['dm.grape7', 'dm.grape5']][idx])

        # Annotation
        ax[idx].annotate(f'{heading}', xy=(0.5, 1.1), xycoords='axes fraction', color='dm.gray9',
                horizontalalignment='center', verticalalignment='bottom', fontsize=dm.fs(-1), fontweight=400)

        # Add value labels on top of each bar
        for bidx, bar in enumerate(bars):
            height = bar.get_height()
            if cidx < 3:
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                f'{height:.0f}', ha='center', va='bottom', fontsize=dm.fs(-2), fontweight = 300)
            elif cidx == 3:
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                f'{height:.1f}', ha='center', va='bottom', fontsize=dm.fs(-2), fontweight = 300)
            
        # Axes limits
        ax[idx].set_xlim(xmin - xmar, xmax + xmar)
        ax[idx].set_ylim(ymin - ymar, ymax + ymar)


        # Tick parameters
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                    which='major', length=0, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                    which='minor', length=0, width=0.25 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                    which='major', length=2, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                    which='minor', length=0, width=0.25 * sp_ratio, pad=3)

        # Labels and ticks
        ax[idx].set_xticks(x_pos)
        ax[idx].set_yticks(np.arange(ymin,ymax+yint,yint))  # No y-ticks
        ax[idx].set_xticklabels(labels, ha='center', fontsize=dm.fs(-1.5), fontweight=300)
        ax[idx].set_yticklabels(np.arange(ymin,ymax+yint,yint), fontsize=dm.fs(-1.5), fontweight= 300)
        
        # Minor ticks
        ax[idx].xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        ax[idx].yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

        # Spines
        for k in ['top','right']:  # Only keep bottom spine
            ax[idx].spines[k].set_visible(False)
        ax[0].spines['bottom'].set_linewidth(0.5 * sp_ratio)
        ax[0].spines['bottom'].set_color('k')

plt.subplots_adjust(wspace=0.4)
dm.simple_layout(fig, bbox=(0.01,1,0.0,1), verbose=False)
img_stem = r'C:\betlab\00 Papers, Conferences\01 Conferences\2025\AIK 춘계학술발표\DHW exergy analysis_HB\figure\GB&HPB_results'
plt.savefig(f'{img_stem}.png', dpi=600)
dm.util.save_and_show(fig)


KeyError: '$E_NG'

# Monte Carlo simulation

## EB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 3,
    'names'   : ['Insulation thick [m]', 'Outdoor air temp [°C]', "Supply water temp [°C]"],
    'bounds'  : [
        [0.01, 0.1]                  ,    # x_ins: 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)] ,            # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)]   ,            # T_sw: 상수도 온도 (°C)
    ],
    'dists': ['unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations)

# 엑서지 효율 저장 리스트
ex_eff_EB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.ElectricBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  
    boiler.T0                 = param_values[i, 1] 
    boiler.T_w_sup            = param_values[i, 2] 

    # 시스템 업데이트 수행
    boiler.system_update()

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w_serv"]
    E_heater = boiler.exergy_balance["hot water tank"]["in"]["E_heater"]

    eta_exergy = X_w_serv / E_heater
    # print(eta_exergy)
    ex_eff_EB.append(eta_exergy)
    
    if eta_exergy < 0:
        print(f"h_co: {boiler.h_co}, eta_exergy: {eta_exergy}")

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 15, 3, 0
ymin, ymax, yint, ymar = 0, 0.2, 0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_EB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        # print(f"y max: {max_frequency}")
        
# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_EB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_EB)*100:.2f} ~ {max(ex_eff_EB)*100:.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

Exergy efficiency range: 4.89 ~ 14.36


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15]],
 'dists': ['unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]'],
 'num_vars': 3,
 'sample_scaled': True}


In [15]:
# Sobol sensitivity analysis
Si = sobol.analyze(problem, np.array(ex_eff_EB), print_to_console=False)

# Visualize first-order sensitivity indices
fig, ax = plt.subplots(figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)

# Bar plot for first-order sensitivity indices
ax.bar(problem['names'], Si['S1'], color='tw.indigo:700', alpha=0.7)

# Add labels and title
ax.set_ylabel('First-order Sensitivity Index', fontsize=dm.fs(0), fontweight=300)
ax.set_xlabel('Input Variables', fontsize=dm.fs(0), fontweight=300)
ax.set_title('Sobol Sensitivity Analysis', fontsize=dm.fs(1), fontweight=400)

# Layout adjustments
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# Save and show the figure
fig_name = 'sobol_sensitivity_EB'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

In [16]:
Si['S1']

array([3.56910006e-05, 9.67816093e-01, 2.98998797e-02])

In [11]:
Si['S2'][0]

array([           nan, 1.46654264e-05, 2.63062850e-06])

## GB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]', 'Outdoor air temp [°C]', "Supply water temp [°C]", "boiler eff [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [0.8, 0.9],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  
}

# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations)

# 엑서지 효율 저장 리스트
ex_eff_GB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.GasBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  # 단열 두께
    boiler.T0                 = param_values[i, 1]  # 대류 열전달 계수
    boiler.T_w_sup            = param_values[i, 2]  # 상수도 온도
    boiler.eta_boiler         = param_values[i, 3]  # 보일러 효율

    # 시스템 업데이트 수행
    boiler.system_update()

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    X_NG = boiler.exergy_balance["combustion chamber"]["in"]["X_NG"]
    
    eta_exergy = X_w_serv / X_NG
    ex_eff_GB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 15, 3, 0
ymin, ymax, yint, ymar = 0, 0.2,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_GB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        # print(f"y max: {max_frequency}")

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_GB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_GB)*100:.2f} ~ {max(ex_eff_GB)*100:.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

Exergy efficiency range: 4.23 ~ 13.73


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15], [0.8, 0.9]],
 'dists': ['unif', 'unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]',
           'boiler eff [-]'],
 'num_vars': 4,
 'sample_scaled': True}


## HPB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]',  'Outdoor air temp [°C]', "Supply water temp [°C]", "COP [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [1.5, 2.5],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성 -> 최종 샘플 개수는 num_simulations * (num_vars + 2) = 10000 * (5 + 2) = 70000
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_HPB = []
cop_list = []
T_r_ext_list = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.HeatPumpBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  # 단열 두께
    boiler.T0                 = param_values[i, 1]  # 대류 열전달 계수
    boiler.T_w_sup            = param_values[i, 2]  # 상수도 온도
    
    # COP calculation
    dT = boiler.T_w_tank - boiler.T0 # 온수 탱크 온도와 실외 온도 차이
    boiler.COP_hp             = 5.06-0.05*dT +0.00006*(dT**2) # https://www.notion.so/betlab/Model-based-flexibility-assessment-of-a-residential-heat-pump-pool-1af6947d125d8037a108e85bb6985dee?pvs=4
    cop_list.append(boiler.COP_hp)
    

    # 시스템 업데이트 수행
    boiler.system_update()
    T_r_ext_list.append(boiler.T_r_ext)

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    E_fan    = boiler.exergy_balance["external unit"]["in"]["E_fan"]
    E_cmp    = boiler.exergy_balance["refrigerant loop"]["in"]["E_cmp"]
    
    eta_exergy = X_w_serv / (E_fan + E_cmp)
    ex_eff_HPB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.15,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
        # print(f"y max: {max_frequency}")
        

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_HPB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
print(f"cop range: {min(cop_list):.2f} ~ {max(cop_list):.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

# # sobol analysis
# Si = sobol.analyze(problem, np.array(ex_eff_HPB), calc_second_order=True, print_to_console=True)

# # 1차 민감도 지수 출력
# print("\n1차 민감도 지수 (S1):")
# for name, s1 in zip(problem["names"], Si["S1"]):
#     print(f"{name}: {s1:.4f}")

Exergy efficiency range: 11.75 ~ 25.23


T_r_ext: -19.9993896484375, -0.0006103515625
Exergy efficiency range: 11.75 ~ 25.23
cop range: 1.85 ~ 2.71


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15], [1.5, 2.5]],
 'dists': ['unif', 'unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]',
           'COP [-]'],
 'num_vars': 4,
 'sample_scaled': True}


In [6]:
print(f"param_values.shape[0]: {param_values.shape[0]}, Y.shape[0]: {len(ex_eff_HPB)}")

param_values.shape[0]: 100000, Y.shape[0]: 100000


## All
- EB, GB, HPB 코드를 모두 실행하여야 돌아감

In [ ]:
# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.2,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(14), dm.cm2in(7)), dpi=600)


# Plot histogram
sns.kdeplot(np.array(ex_eff_EB) * 100, ax=ax, color='tw.orange:700', fill=True, linewidth=0.75)
sns.kdeplot(np.array(ex_eff_GB) * 100, ax=ax, color='tw.pink:700', fill=True, linewidth=0.75)
sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

# tick limits 
ax.set_xlim(xmin - xmar, xmax + xmar)
ax.set_ylim(ymin - ymar, ymax + ymar)

# tick parameters
ax.tick_params(axis='x', labelsize=dm.fs(1), which='major')
ax.tick_params(axis='y', labelsize=dm.fs(1), which='major')

# set ticks
ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
ax.set_yticks(np.arange(ymin, ymax + yint, yint))

# legend 
ax.legend(['Electric boiler', 'Gas boiler', 'Heat pump boiler'], fontsize=dm.fs(0),)

# print x, y range
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
# print(f"y max: {max_frequency}")


plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(2), fontweight=300 )
plt.ylabel("Probability density", fontsize=dm.fs(2), fontweight=300)
# plt.title("Exergy Efficiency Distribution from Monte Carlo Simulation", fontsize=dm.fs(2), fontweight=400)
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)
fig_name = 'ex_eff_distribution_all'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

Exergy efficiency range: 11.75 ~ 25.23


## ASHP

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]',  'Outdoor air temp [°C]', "Supply water temp [°C]", "COP [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [1.5, 2.5],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성 -> 최종 샘플 개수는 num_simulations * (num_vars + 2) = 10000 * (5 + 2) = 70000
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_HPB = []
cop_list = []
T_r_ext_list = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricASHP 객체 생성
    ASHP = enex.HeatPumpASHP()

    # 확률 변수에서 추출한 변수 값들 할당
    ASHP.x_ins              = param_values[i, 0]  # 단열 두께
    ASHP.T0                 = param_values[i, 1]  # 대류 열전달 계수
    ASHP.T_w_sup            = param_values[i, 2]  # 상수도 온도
    
    # COP calculation
    dT = ASHP.T_w_tank - ASHP.T0 # 온수 탱크 온도와 실외 온도 차이
    ASHP.COP_hp             = 5.06-0.05*dT +0.00006*(dT**2) # https://www.notion.so/betlab/Model-based-flexibility-assessment-of-a-residential-heat-pump-pool-1af6947d125d8037a108e85bb6985dee?pvs=4
    
    
    cop_list.append(ASHP.COP_hp)
    

    # 시스템 업데이트 수행
    ASHP.system_update()
    T_r_ext_list.append(ASHP.T_r_ext)

    # 엑서지 효율 계산
    X_w_serv = ASHP.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    E_fan    = ASHP.exergy_balance["external unit"]["in"]["E_fan"]
    E_cmp    = ASHP.exergy_balance["refrigerant loop"]["in"]["E_cmp"]
    
    eta_exergy = X_w_serv / (E_fan + E_cmp)
    ex_eff_HPB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.15,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
        # print(f"y max: {max_frequency}")
        

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_HPB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
print(f"cop range: {min(cop_list):.2f} ~ {max(cop_list):.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")

In [ ]:
# Sobol sensitivity analysis
Si = sobol.analyze(problem, np.array(ex_eff_EB), print_to_console=False)

# Visualize first-order sensitivity indices
fig, ax = plt.subplots(figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)

# Bar plot for first-order sensitivity indices
ax.bar(problem['names'], Si['S1'], color='tw.indigo:700', alpha=0.7)

# Add labels and title
ax.set_ylabel('First-order Sensitivity Index', fontsize=dm.fs(0), fontweight=300)
ax.set_xlabel('Input Variables', fontsize=dm.fs(0), fontweight=300)
ax.set_title('Sobol Sensitivity Analysis', fontsize=dm.fs(1), fontweight=400)

# Layout adjustments
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# Save and show the figure
fig_name = 'sobol_sensitivity_EB'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

# Test